In [12]:
import pandas as pd
import numpy as np
import random # used to randomly choose a word from list of words
import string # needed to remove punctation from words

In [9]:
def remove_punc(word, punc_set = string.punctuation):
  return ''.join(ch for ch in word if ch not in punc_set)

Preprocess File

In [45]:
# get the text from the file
def gen_corpus_from_file(in_file="/content/nursery_rhymes.txt", 
                         out_file="/content/parsed_txt.txt"):
  headers_lst = []
  rhymes_lst = []
  skip_ln = lambda ln : True if (ln == "") else False
  is_header = lambda ln : True if (ln[-1].isupper()) else False
  with open(in_file) as file:
    for line in file:
      strip_line = line.replace('\r', ' ').replace('\n', ' ').strip()
      if skip_ln(strip_line): continue
      if is_header(strip_line):
        headers_lst.append(strip_line)
      else:
        rhyme = remove_punc(strip_line.lower())
        rhymes_lst.append(rhyme)
  rhymes = ' '.join(rhymes_lst)
  del headers_lst
  del rhymes_lst
  with open(out_file, "w") as out_f:
    out_f.write(rhymes)
  return rhymes.split(' ')

In [46]:
words = gen_corpus_from_file()

# Show count of many words there are
print(f'Corpus size: {len(words)} words.')
# print a few words to show parsing worked
print(words[slice(10)])

Corpus size: 17600 words.
['the', 'queen', 'of', 'hearts', 'she', 'made', 'some', 'tarts', 'all', 'on']


Build the transition probabilities


*   Represent transitions as a dictionary where the keys are the distinct words and the value is the list of words that appear after that respective key


1st order Markov Chain method

In [ ]:
def gen_1st_order_chain(words):
  chain = {}  
  n_words = len(words)  
  for i, key in enumerate(words):  
      if n_words > (i + 1):
          word = words[i + 1]
          if key not in chain:
              chain[key] = [word]
          else:
              chain[key].append(word)
  return chain

In [ ]:
chain = gen_1st_order_chain(words)
print('Chain size: {0} distinct words.'.format(len(chain)))

Chain size: 2445 distinct words.


Why duplicates are not a problem:

*   If a word appears multiple times in the list, and take a random sample from the list during a transition, there’s a higher likelihood that word would be picked proportional to the number of times it appeared after the key relative to all the other words in the corpus that appeared after that key



Generating rhyme from the Markov Chain requires only a starting word and a phrase length. To generate a phrase:


1.   Randomly select a starting word from the corpus
2.   Make rhyme length 20 words long
3.   Repeat until have 30 lines of these.





First order Markov Chain rhyme method

In [ ]:
# First order rhyme generator
def gen_1st_order_rhyme(words, chain):
  rand_rhyme = ""
  rand_word_1 = random.choice(words)
  rand_rhyme = rand_word_1  
  rhyme = []
  rhyme.append(rand_word_1)
  for i in range(19):  
      rand_word_2 = random.choice(chain[rand_word_1])
      rhyme.append(rand_word_2)
      rand_rhyme += ' ' + rand_word_2
      rand_word_1 = rand_word_2
  print(rhyme)
  print(f"There are {len(rhyme)} many words.")
  return rand_rhyme

In [ ]:
print(gen_1st_order_rhyme(words, chain))

['a', 'strange', 'sight', 'upon', 'you', 'shoe', 'three', 'four', 'and', 'a', 'rowley', 'powley', 'gammon', 'and', 'hear', 'the', 'cat', 'that', 'ate', 'the']
There are 20 many words.
a strange sight upon you shoe three four and a rowley powley gammon and hear the cat that ate the


2nd order Markov Chain method

In [ ]:
# First order rhyme generator
def gen_2nd_order_chain(words):
  chain = {}  
  n_words = len(words)  
  for i, key1 in enumerate(words):  
      if n_words > i + 2:
          key2 = words[i + 1]
          word = words[i + 2]
          if (key1, key2) not in chain:
              chain[(key1, key2)] = [word]
          else:
              chain[(key1, key2)].append(word)
  return chain

In [ ]:
chain_2 = gen_2nd_order_chain(words)
print(f'Chain size: {len(chain_2)} distinct word pairs.')

Chain size: 10202 distinct word pairs.


Choosing a word pair that appears somewhere in the text and then examining the transitions in the chain for that pair of words (to ensure that the 2nd order chain is operating correctly).

In [ ]:
# 'to your' appears in nusery_rhymes.txt multiple times
chain_2[("to", "your")]

['daddie',
 'daddie',
 'daddie',
 'daddie',
 'daddie',
 'mammie',
 'playfellows',
 'sons']

2nd order Markov Chain rhyme method

In [ ]:
def gen_2nd_order_rhyme(chain, words):
    rhyme_lst = []  
    rand_word = random.randint(0, len(words) - 1)
    rhyme_lst.append(rand_word)
    key = (words[rand_word], words[rand_word + 1])
    rhyme = key[0] + ' ' + key[1]
    for i in range(19):
        rand_word_2 = random.choice(chain[key])
        rhyme_lst.append(rand_word_2)
        rhyme += ' ' + rand_word_2
        key = (key[1], rand_word_2)
    return rhyme

In [ ]:
# testing 2nd order method
for i in range(4): print(gen_2nd_order_rhyme(chain_2, words))

black sheep have you seen sam and betsy on the wall and thats half he took him out of france came
pig with a ring a ting tang poor old robinson crusoe poor old robinson crusoe poor old robinson crusoe poor old
he said nay the third he said nay the third he said little maid will you have caught cold mr frog
on snails ho ho quoth the iron till it would not do his master came in and began to quarrel and


## Creating MarkovChain Class Model

In [47]:
class MarkovChain():
  def __init__(self, order, words):
    self.order = order
    self.words = words
    self.chain = {}
    self.created_chain = False
    if self.order == 1:
      self.gen_1st_order_chain()
      self.created_chain = True
    elif self.order == 2:
      self.gen_2nd_order_chain()
      self.created_chain = True
  def reset_chain(self):
    self.chain = {}
    self.order = None
    self.created_chain = False
  def set_order(self, order):
    self.order = order
  # sets chain to a 1st order Markov Chain
  def gen_1st_order_chain(self):
    if self.order != 1:
      print(f"Error: Cannot create \
      a 1st order chain when order is set to {self.order}")
      return
    if self.created_chain:
      print(f"Error: A {self.order} chain is \
      already made. Reset chain (and order) \
      before creating new chain.")
      return
    n_words = len(self.words)  
    for i, key in enumerate(self.words):  
        if n_words > (i + 1):
            word = self.words[i + 1]
            if key not in self.chain:
                self.chain[key] = [word]
            else:
                self.chain[key].append(word)
  # 1st order rhyme generator
  def gen_1st_order_rhyme(self):
    if self.order != 1:
      print(f"Error: Cannot produce \
      a 1st order rhyme when order is set to {self.order}")
      return
    rand_rhyme = ""
    rand_word_1 = random.choice(self.words)
    rand_rhyme = rand_word_1  
    rhyme = []
    rhyme.append(rand_word_1)
    for i in range(19):  
        rand_word_2 = random.choice(self.chain[rand_word_1])
        rhyme.append(rand_word_2)
        rand_rhyme += ' ' + rand_word_2
        rand_word_1 = rand_word_2
    return rand_rhyme
  # sets chain to a 2nd order Markov Chain
  def gen_2nd_order_chain(self):
    if self.order != 2:
      print(f"Error: Cannot produce \
      a 2nd order chain when order is set to {self.order}")
      return
    if self.created_chain:
      print(f"Error: A {self.order} chain is \
      already made. Reset chain (and order) \
      before creating new chain.")
      return
    n_words = len(self.words)  
    for i, key1 in enumerate(self.words):  
        if n_words > i + 2:
            key2 = self.words[i + 1]
            word = self.words[i + 2]
            if (key1, key2) not in self.chain:
                self.chain[(key1, key2)] = [word]
            else:
                self.chain[(key1, key2)].append(word)
  # 2nd order rhyme generator
  def gen_2nd_order_rhyme(self):
    if self.order != 2:
      print(f"Error: Cannot produce \
      a 2nd order rhyme when order is set to {self.order}")
      return
    rhyme_lst = []  
    rand_word = random.randint(0, len(self.words) - 1)
    rhyme_lst.append(rand_word)
    key = (self.words[rand_word], self.words[rand_word + 1])
    rhyme = key[0] + ' ' + key[1]
    for i in range(19):
        rand_word_2 = random.choice(self.chain[key])
        rhyme_lst.append(rand_word_2)
        rhyme += ' ' + rand_word_2
        key = (key[1], rand_word_2)
    return rhyme

In [48]:
markov_model = MarkovChain(2, words)

In [49]:
def gen_nusery_rhymes(markov_model, n_lines=30, output_file="markov_rhymes.txt"):
  with open(output_file, "w") as out_file:
    for i in range(n_lines):
      curr_line = markov_model.gen_2nd_order_rhyme()
      out_file.write(curr_line + ".\n")

In [50]:
gen_nusery_rhymes(markov_model)

Sources:
*   Medium article by John Wittenauer: https://medium.com/@jdwittenauer/markov-chains-from-scratch-33340ba6535b



---

